In [1]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.time_series_functions import *
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

# import tensorflow as tf
# keras = tf.keras
# from keras.preprocessing.sequence import TimeseriesGenerator
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.callbacks import EarlyStopping#, ModelCheckpoint
# from keras.metrics import MeanAbsolutePercentageError


%matplotlib inline

Using TensorFlow backend.


In [2]:
#load in clean master dataframe

In [4]:
base_df = csv_with_datetime('data/combdf_snoflow_and_cfs1.csv.zip', 'Unnamed: 0')
df = base_df.copy()

In [5]:
#small % of records containing NaNs in the temp columns, randomly distributed. using ffill. 
df.fillna(method='ffill', inplace=True)

In [6]:
#keeping only the columns we want, and renaming them to more convenient names
df = df[['ds', 'Streamflow Value', 'Air Temperature Average (degF)',\
                'Precipitation Accumulation (in) Start of Day Values',\
                'Snow Water Equivalent (in) Start of Day Values']]

df.columns = ['ds', 'y', 'avg_temp', 'precip_accum', 'swe']

#cleaning the dataframe to end at 12/31/2020 for cleaner working frame
df = df['1990':'2020']
#we now have a full 31 year dataframe (01/01/1990 - 12/31/2020), no nans, and columns appropriately named
#granularity of the data is daily, writing code based on day count not calendar year for OOP

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11323 entries, 1990-01-01 to 2020-12-31
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            11323 non-null  datetime64[ns]
 1   y             11323 non-null  float64       
 2   avg_temp      11323 non-null  float64       
 3   precip_accum  11323 non-null  float64       
 4   swe           11323 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 530.8 KB


In [37]:
#creating a dataframe to represent historical average for each feature for each day of the year
historical_avg = df.groupby([df.index.month, df.index.day]).mean()
#pull dates from leap year to align with averages, add 'ds' column and set as index for 366 day dataframe


In [40]:
dates_2020 = df['ds'].copy()



In [54]:
dates_2020 = dates_2020['2020']

In [55]:
dates_2020 = pd.DataFrame(dates_2020)

In [58]:
dates_2020['y'] = historical_avg['y'].values

In [62]:
dates_2020.head(3)

,ds,y
Unnamed: 0,,
2020-01-01,2020-01-01,6.437742
2020-01-02,2020-01-02,6.502258
2020-01-03,2020-01-03,6.673226


In [63]:
dates_2020['string'] = dates_2020['ds'].dt.strftime('%m-%d')

In [69]:
hist_avg = dates_2020.copy()

In [71]:
hist_avg.set_index('string', drop=False, inplace=True)

In [72]:
hist_avg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 366 entries, 01-01 to 12-31
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      366 non-null    datetime64[ns]
 1   y       366 non-null    float64       
 2   string  366 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 11.4+ KB


string
01-01    6.437742
01-02    6.502258
01-03    6.673226
01-04    6.738387
01-05    6.955806
           ...   
12-27    7.052581
12-28    6.748710
12-29    6.567419
12-30    6.378387
12-31    6.437742
Name: y, Length: 366, dtype: float64

In [65]:
dates_2019 = df['2019'].copy()

In [67]:
dates_2019['string'] = dates_2019['ds'].dt.strftime('%m-%d')

In [79]:
dates_2019 = dates_2019.merge(hist_avg['y'], how='left', on='string', )

In [83]:
dates_2019.set_index('ds', drop=False, inplace=True)

In [85]:
work_df = df.copy()

In [86]:
work_df['string'] = work_df['ds'].dt.strftime('%m-%d')

In [87]:
work_df = work_df.merge(hist_avg['y'], how='left', on='string', )

In [88]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11323 entries, 0 to 11322
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            11323 non-null  datetime64[ns]
 1   y_x           11323 non-null  float64       
 2   avg_temp      11323 non-null  float64       
 3   precip_accum  11323 non-null  float64       
 4   swe           11323 non-null  float64       
 5   string        11323 non-null  object        
 6   y_y           11323 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 707.7+ KB


In [91]:
work_df.tail()

,ds,y_x,avg_temp,precip_accum,swe,string,y_y
11318,2020-12-27,6.23,13.0,4.9,4.8,12-27,7.052581
11319,2020-12-28,5.80,13.0,5.0,4.9,12-28,6.748710
11320,2020-12-29,4.99,9.0,5.4,5.3,12-29,6.567419
11321,2020-12-30,4.48,6.0,5.4,5.3,12-30,6.378387
11322,2020-12-31,5.62,16.0,5.5,5.4,12-31,6.437742


In [92]:
work_df.columns = ['ds', 'y', 'avg_temp', 'precip_accum', 'swe', 'string', 'hist_avg_y']

In [93]:
work_df.tail()

,ds,y,avg_temp,precip_accum,swe,string,hist_avg_y
11318,2020-12-27,6.23,13.0,4.9,4.8,12-27,7.052581
11319,2020-12-28,5.80,13.0,5.0,4.9,12-28,6.748710
11320,2020-12-29,4.99,9.0,5.4,5.3,12-29,6.567419
11321,2020-12-30,4.48,6.0,5.4,5.3,12-30,6.378387
11322,2020-12-31,5.62,16.0,5.5,5.4,12-31,6.437742


In [94]:
work_df.drop('string', axis=1, inplace=True)

In [95]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11323 entries, 0 to 11322
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            11323 non-null  datetime64[ns]
 1   y             11323 non-null  float64       
 2   avg_temp      11323 non-null  float64       
 3   precip_accum  11323 non-null  float64       
 4   swe           11323 non-null  float64       
 5   hist_avg_y    11323 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 619.2 KB


In [96]:
work_df.to_csv('data/master_df.csv')

In [2]:
base_df = csv_with_datetime('data/master_df.csv', 'ds')
df = base_df.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11323 entries, 1990-01-01 to 2020-12-31
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    11323 non-null  int64         
 1   ds            11323 non-null  datetime64[ns]
 2   y             11323 non-null  float64       
 3   avg_temp      11323 non-null  float64       
 4   precip_accum  11323 non-null  float64       
 5   swe           11323 non-null  float64       
 6   hist_avg_y    11323 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 707.7 KB


In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.head()

,ds,y,avg_temp,precip_accum,swe,hist_avg_y
ds,,,,,,
1990-01-01,1990-01-01,6.5,19.0,5.6,4.8,6.437742
1990-01-02,1990-01-02,6.9,16.0,5.7,4.8,6.502258
1990-01-03,1990-01-03,6.9,2.0,5.7,4.8,6.673226
1990-01-04,1990-01-04,6.8,-2.0,5.7,4.8,6.738387
1990-01-05,1990-01-05,7.3,1.0,5.7,4.8,6.955806
